In [1]:
import os
import sys
sys.path.append(r'/efs/Incomati/scripts_aridity')
import pandas as pd
import time

from WAsheets import calculate_flux as cf
from WAsheets import hydroloop as hl
from WAsheets import sheet2_v3 as sheet2
from WAsheets import print_sheet as ps

In [ ]:
wp_folder=r"/efs/CWA/ncdf_files_sheet2_07_11/" #folder with nc files from water balance model
BASIN={
       'name': 'AFR',
       'hydroyear':'A-DEC', #Water year end month
       'chunksize':[1, 1000, 1000],
       'unit_conversion':1e6, #1e3 for input data in MCM, 1e6 for input data in km3
       'output_folder':r"/efs/CWA/hydroloop_results_07_11_2_test",
       'gis_data':{
               'basin_mask': r'/efs/CWA/static_datasets/basin_mask/AFR_basinmask_1km.tif',
               'subbasin_mask':{
                       1: r"/efs/CWA/static_datasets/basin_mask/AFR_basinmask_1km.tif"                       
                       },
               },
       'data_cube':{
           'monthly':{
               'eta':os.path.join(wp_folder, 'eta_monthly_added.nc'),
               'nrd':os.path.join(wp_folder, 'nRD_monthly.nc'),
               'ndm':os.path.join(wp_folder, 'AFR_NDM_ProbaV.nc'),
               'lai':os.path.join(wp_folder, 'AFR_LAI_MOD15.nc'),
               'lu':os.path.join(wp_folder, 'AFR_LU_WA_resampled_monthly.nc'),
               'p':os.path.join(wp_folder, 'AFR_P_CHIRPS.nc'),
               'i':os.path.join(wp_folder, 'i_monthly.nc')
               },
           'yearly':{
                'lu':os.path.join(wp_folder,'AFR_LU_WA.nc'),
                   }              
       }
}


start = time.time()

out_folder = BASIN['output_folder']
if not os.path.exists(out_folder):
    os.makedirs(out_folder)
    
#%% Calculate yearly and (intermediate) monthly sheet csvs

sheet2_yearly_csvs=sheet2.main(BASIN,unit_conversion=BASIN['unit_conversion'])



if BASIN['unit_conversion'] == 1e6:
    str_unit='km3/year'
elif BASIN['unit_conversion']==1e3:
    str_unit='MCM/year'
    
for sheet2_csv in sheet2_yearly_csvs:
    period=os.path.basename(sheet2_csv).split('.')[0].split('_')[-1]
    output=sheet2_csv.replace('.csv','.pdf')
    ps.print_sheet2(BASIN['name'],period=period,
                    output=output,units=str_unit,data=sheet2_csv)
    
    
end = time.time()
total_time = end - start
print ('total time :',total_time)

In [ ]:
BASIN['data_cube']['monthly']['eta']